## Install the required dependencies

In [ ]:
# Install the required dependencies
! pip install haystack-ai>=2.14.0
! pip install docstring-parser



Next, we configure our variables. We need to set the `OPENAI_API_KEY` for [`OpenAIChatGenerator`](https://docs.haystack.deepset.ai/docs/openaichatgenerator).

In [1]:
from getpass import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key:")


Create the tools to get user input and pass it on to next component.

In [2]:
from typing import Annotated
from haystack.tools import tool, ComponentTool
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack.components.generators.utils import print_streaming_chunk

@tool
def human_in_loop_tool(
    question: Annotated[
        str,
        "A clarifying question to prompt the user for more information"
    ]
) -> str:
    '''Ask the user a clarifying question and return their response via input().'''
    user_message = input(f"[Agent needs your input] {question}\n> ")
    return user_message

@tool
def hand_off_to_next_tool(
    info_package: Annotated[
        dict,
        "The information package to be passed to the downstream agent or terminal"
    ]
) -> dict:
    """
    Stub: returns the info_package so that your orchestrator
    (or whichever process is listening) can pick it up.
    """
    return info_package


## Create and Configure the Need Analysis Agent
Create a Haystack Agent and configure its behavior with a system prompt and tools. Then we convert it into a `ComponentTool` to be used by another agent.

In [3]:
need_analysis_agent = Agent(
    chat_generator=OpenAIChatGenerator(model="o4-mini"),
    tools=[human_in_loop_tool, hand_off_to_next_tool],
    streaming_callback=print_streaming_chunk,
    system_prompt=(
        "You are a Needs‐Analysis Research Agent. Your sole job is to elicit from the user all the facts, context, constraints, "
        "and preferences necessary to solve their query. Treat each new user query as a requirement gathering exercise. Ask only one question in a single message to prevent confusion.\n\n"
        "1. **Clarify the Goal**: Begin by restating the user’s request in your own words. Ask follow-ups to ensure you understand their high-level objective.\n"
        "2. **Uncover Context**: Ask about background, motivations, timeline, budget/resources, stakeholders, technical constraints or any existing work.\n"
        "3. **Elicit Preferences**: Drill down into style preferences, priorities (e.g. cost vs. quality vs. speed), formats, platforms, level of detail, etc.\n"
        "4. **Validate and Summarize**: Periodically summarize what you’ve heard and ask the user to confirm or correct. This ensures completeness.\n"
        "5. **Depth and Edge Cases**: Probe for exceptions, unusual scenarios, or potential trade-offs the user may not have considered.\n\n"
        "**Exit Condition**: Call the `hand_off_to_next_tool` and exit when you have gathered a clear, structured “information package” (objective + context + constraints + preferences) "
        "to enable a downstream agent to produce a full solution."
    ),
    exit_conditions=["hand_off_to_next_tool"]
)

need_analysis_agent.warm_up()

need_analysis_tool = ComponentTool(
    component=need_analysis_agent,
    description="Use this tool to gather information about the user's needs and preferences",
    name="need_analysis_tool"
)


For this demo, we use a `consulting_agent` that calls the `need_analysis_tool` to gather user requirements and preferences, then uses that information to generate a comprehensive recommendation. There is only tool for now to keep it simple, later we can add more tools.

In [6]:
consulting_agent = Agent(
    chat_generator=OpenAIChatGenerator(model="o4-mini", streaming_callback=print_streaming_chunk),
    system_prompt="""
    You are a car consultant agent that generates a car recommendation based on the user's needs and preferences.
    """,
    tools=[need_analysis_tool],
    exit_conditions=["text"]
)

consulting_agent.warm_up()





## See the Agent in Action

In [7]:
message=[ChatMessage.from_user("I want to buy a Range Rover ")]
ans = consulting_agent.run(messages=message)



[TOOL CALL]
Tool: need_analysis_tool 
Arguments: {"messages":[{"role":"user","content":[{"text":"I want to buy a Range Rover"}]}]}



[TOOL CALL]
Tool: human_in_loop_tool 
Arguments: {"question":"It sounds like you're looking to purchase a Range Rover. Could you tell me what your main objectives are for this vehicle—for example, how you plan to use it, what features you consider essential, and any budget or timeline constraints?"}



[TOOL RESULT]
I need the car for daily commute and occasional trips



[TOOL CALL]
Tool: human_in_loop_tool 
Arguments: {"question":"Understood. To help narrow down the right Range Rover model and configuration, what is your budget range for the purchase and do you have a specific timeframe or deadline by which you need the vehicle?"}



[TOOL RESULT]
budegt is 100k, I want to buy the car next month



[TOOL CALL]
Tool: human_in_loop_tool 
Arguments: {"question":"Thanks for clarifying. Next, do you prefer a brand-new Range Rover or would you consider a c

Failed to invoke Tool `hand_off_to_next_tool` with parameters {}. Error: hand_off_to_next_tool() missing 1 required positional argument: 'info_package'






[TOOL CALL]
Tool: hand_off_to_next_tool 
Arguments: {"info_package":{"objective":"Purchase a new Range Rover for daily commuting and occasional trips","context":{"use_case":"Daily commute plus occasional trips"},"constraints":{"budget":100000,"purchase_timeline":"Within next month","model_condition":"Brand new"},"preferences":{"model_options":["Sport","Evoque"],"seating_capacity":5,"powertrain":"Hybrid engine","roof":"Panoramic","performance":"High horsepower","exterior_color":"Black","interior_material":"Leather","driver_assistance":["Adaptive cruise control"]}}}



[TOOL RESULT]
{'objective': 'Purchase a new Range Rover for daily commuting and occasional trips', 'context': {'use_case': 'Daily commute plus occasional trips'}, 'constraints': {'budget': 100000, 'purchase_timeline': 'Within next month', 'model_condition': 'Brand new'}, 'preferences': {'model_options': ['Sport', 'Evoque'], 'seating_capacity': 5, 'powertrain': 'Hybrid engine', 'roof': 'Panoramic', 'performance': 'High 